# 1. Import libraries

In [1]:
#----------------------------Reproducible----------------------------------------------------------------------------------------
import numpy as np
import tensorflow as tf
import random as rn
import os

seed=0
os.environ['PYTHONHASHSEED'] = str(seed)

np.random.seed(seed)
rn.seed(seed)

#----------------------------Reproducible----------------------------------------------------------------------------------------

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

#--------------------------------------------------------------------------------------------------------------------------------
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline
matplotlib.style.use('ggplot')

import random
import scipy.sparse as sparse
import scipy.io
import time
from keras.utils import to_categorical
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from skfeature.utility import construct_W
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from skfeature.utility.sparse_learning import feature_ranking
import time
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from concrete_autoencoder import ConcreteAutoencoderFeatureSelector
from keras.layers import Dense
import pandas as pd
from skimage import io
from PIL import Image
from sklearn.model_selection import train_test_split
import scipy.sparse as sparse
from keras.datasets import fashion_mnist
import scipy.io

#--------------------------------------------------------------------------------------------------------------------------------
#Import ourslef defined methods
import sys
sys.path.append(r"../Defined")
import Functions as F

Using TensorFlow backend.


# 2. Loading data

In [2]:
data_path="../Dataset/arcene.mat"
Data = scipy.io.loadmat(data_path)

data_arr=Data['X']
label_arr_=Data['Y'][:, 0]

label_arr=np.array([0 if label_arr_i<0 else label_arr_i for label_arr_i in label_arr_])
data_arr=MinMaxScaler(feature_range=(0,1)).fit_transform(data_arr)
label_arr_onehot=label_arr

In [3]:
key_feture_number=64

# 3. Calculation

In [4]:
#--------------------------------------------------------------------------------------------------------------------------------
def write_to_csv(p_data,p_path):
    dataframe = pd.DataFrame(p_data)
    dataframe.to_csv(p_path, mode='a',header=False,index=False,sep=',')
    del dataframe

#--------------------------------------------------------------------------------------------------------------------------------       
def mse_check(train, test):
    LR = LinearRegression(n_jobs = -1)
    LR.fit(train[0], train[1])
    MSELR = ((LR.predict(test[0]) - test[1]) ** 2).mean()
    return MSELR
 
#--------------------------------------------------------------------------------------------------------------------------------       
def cal(p_data_arr,\
        p_label_arr_onehot,\
        p_key_feture_number,\
        p_epochs_number,\
        p_seed):
    
    C_train_x,C_test_x,C_train_y,C_test_y= train_test_split(p_data_arr,p_label_arr_onehot,test_size=0.2,random_state=p_seed)

    os.environ['PYTHONHASHSEED'] = str(p_seed)
    np.random.seed(p_seed)
    rn.seed(p_seed)
    tf.compat.v1.set_random_seed(p_seed)
    
    #--------------------------------------------------------------------------------------------------------------------------------
    
    def decoder(x):
        #x = Dense(key_feture_number)(x)
        x = Dense(data_arr.shape[1])(x)
        return x

    t_start = time.time()
    selector = ConcreteAutoencoderFeatureSelector(K = p_key_feture_number, output_function = decoder, num_epochs = p_epochs_number)
    selector.fit(C_train_x, C_train_x, C_test_x, C_test_x)
    t_used=time.time() - t_start
    
    write_to_csv(np.array([t_used]),"./log/CAE_time.csv")
    
    train_compressed_Data=p_data_arr[:, selector.get_support(indices=True)]
    
    # Classification on original features
    train_feature=C_train_x
    train_label=C_train_y
    test_feature=C_test_x
    test_label=C_test_y 
    orig_train_acc,orig_test_acc=F.ETree(train_feature,train_label,test_feature,test_label,0)
        
    # Classification on selected features
    C_train_selected_x,C_test_selected_x,C_train_y,C_test_y= train_test_split(train_compressed_Data,p_label_arr_onehot,test_size=0.2,random_state=p_seed)
    
    train_feature=C_train_selected_x
    train_label=C_train_y
    test_feature=C_test_selected_x
    test_label=C_test_y
    selec_train_acc,selec_test_acc=F.ETree(train_feature,train_label,test_feature,test_label,0)
    
    # Linear reconstruction
    train_feature_tuple=(C_train_selected_x,C_train_x)
    test_feature_tuple=(C_test_selected_x,C_test_x)

    reconstruction_loss=mse_check(train_feature_tuple, test_feature_tuple)
    
    results=np.array([orig_train_acc,orig_test_acc,selec_train_acc,selec_test_acc,reconstruction_loss])
    
    write_to_csv(results.reshape(1,len(results)),"./log/CAE_results.csv")
    
    return orig_train_acc,orig_test_acc,selec_train_acc,selec_test_acc,reconstruction_loss

In [5]:
epochs_number=200

p_data_arr=data_arr
p_label_arr_onehot=label_arr_onehot
p_key_feture_number=key_feture_number
p_epochs_number=epochs_number

In [ ]:
for p_seed in np.arange(0,50):
    orig_train_acc,orig_test_acc,selec_train_acc,selec_test_acc,reconstruction_loss=cal(p_data_arr,\
                                                                                        p_label_arr_onehot,\
                                                                                        p_key_feture_number,\
                                                                                        p_epochs_number,\
                                                                                        p_seed)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10000)             0         
_________________________________________________________________
concrete_select (ConcreteSel (None, 64)                640001    
_________________________________________________________________
dense_1 (Dense)              (None, 10000)             650000    
Total params: 1,290,001
Trainable params: 1,290,000
Non-trainable params: 1
_________________________________________________________________
None

Train on 160 samples, validate on 40 samples
Epoch 1/200
mean max of probabilities: 0.0001032453 - temperature 10.0
160/160 [==============================] - 4s 23ms/step - loss: 0.0851 - val_loss: 0.0711
Epoch 2/200
mean max of probabilities: 0.000103245344 - temperature 9.705095
160/160 [

/usr/local/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.186670). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


160/160 [==============================] - 2s 13ms/step - loss: 0.0415 - val_loss: 0.0428
Epoch 38/200
mean max of probabilities: 0.00010336548 - temperature 3.3036494
160/160 [==============================] - 1s 9ms/step - loss: 0.0414 - val_loss: 0.0426
Epoch 39/200
mean max of probabilities: 0.000103383616 - temperature 3.2062237
160/160 [==============================] - 7s 41ms/step - loss: 0.0414 - val_loss: 0.0427
Epoch 40/200
mean max of probabilities: 0.00010340365 - temperature 3.1116705
160/160 [==============================] - 2s 11ms/step - loss: 0.0414 - val_loss: 0.0426
Epoch 41/200
mean max of probabilities: 0.00010342453 - temperature 3.0199058
160/160 [==============================] - 1s 6ms/step - loss: 0.0413 - val_loss: 0.0425
Epoch 42/200
mean max of probabilities: 0.00010344837 - temperature 2.9308476
160/160 [==============================] - 1s 9ms/step - loss: 0.0412 - val_loss: 0.0424
Epoch 43/200
mean max of probabilities: 0.00010347305 - temperature 2.84

/usr/local/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.146264). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


160/160 [==============================] - 1s 7ms/step - loss: 0.0408 - val_loss: 0.0423
Epoch 52/200
mean max of probabilities: 0.00010383836 - temperature 2.172659
160/160 [==============================] - 1s 5ms/step - loss: 0.0408 - val_loss: 0.0422
Epoch 53/200
mean max of probabilities: 0.00010389711 - temperature 2.108586
160/160 [==============================] - 1s 4ms/step - loss: 0.0409 - val_loss: 0.0418
Epoch 54/200
mean max of probabilities: 0.00010395898 - temperature 2.0464027
160/160 [==============================] - 2s 15ms/step - loss: 0.0408 - val_loss: 0.0420
Epoch 55/200
mean max of probabilities: 0.00010402643 - temperature 1.9860538
160/160 [==============================] - 3s 17ms/step - loss: 0.0407 - val_loss: 0.0420
Epoch 56/200
mean max of probabilities: 0.00010409481 - temperature 1.9274842
160/160 [==============================] - 4s 22ms/step - loss: 0.0407 - val_loss: 0.0420
Epoch 57/200
mean max of probabilities: 0.00010416588 - temperature 1.87064

...

160/160 [==============================] - 2s 12ms/step - loss: 0.0383 - val_loss: 0.0406
Epoch 178/200
mean max of probabilities: 0.00011284849 - temperature 0.1
160/160 [==============================] - 1s 9ms/step - loss: 0.0389 - val_loss: 0.0417
Epoch 179/200
mean max of probabilities: 0.00011285086 - temperature 0.1
160/160 [==============================] - 2s 11ms/step - loss: 0.0382 - val_loss: 0.0398
Epoch 180/200
mean max of probabilities: 0.000112855065 - temperature 0.1
160/160 [==============================] - 1s 8ms/step - loss: 0.0381 - val_loss: 0.0390
Epoch 181/200
mean max of probabilities: 0.00011288797 - temperature 0.1
160/160 [==============================] - 1s 9ms/step - loss: 0.0391 - val_loss: 0.0399
Epoch 182/200
mean max of probabilities: 0.00011290173 - temperature 0.1
160/160 [==============================] - 1s 5ms/step - loss: 0.0394 - val_loss: 0.0397
Epoch 183/200
mean max of probabilities: 0.000112920214 - temperature 0.1
160/160 [===============

/usr/local/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.314803). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


160/160 [==============================] - 16s 98ms/step - loss: 0.0411 - val_loss: 0.0414
Epoch 108/200
mean max of probabilities: 0.00011091444 - temperature 0.40642717
160/160 [==============================] - 6s 36ms/step - loss: 0.0413 - val_loss: 0.0412
Epoch 109/200
mean max of probabilities: 0.00011098445 - temperature 0.3944415
160/160 [==============================] - 6s 37ms/step - loss: 0.0412 - val_loss: 0.0406
Epoch 110/200
mean max of probabilities: 0.00011103123 - temperature 0.38280922
160/160 [==============================] - 8s 53ms/step - loss: 0.0400 - val_loss: 0.0410
Epoch 111/200
mean max of probabilities: 0.00011109025 - temperature 0.37151998
160/160 [==============================] - 24s 153ms/step - loss: 0.0416 - val_loss: 0.0416
Epoch 112/200
mean max of probabilities: 0.000111155845 - temperature 0.36056373
160/160 [==============================] - 15s 95ms/step - loss: 0.0407 - val_loss: 0.0390
Epoch 113/200
mean max of probabilities: 0.00011121972 -

/usr/local/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.234892). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


160/160 [==============================] - 8s 49ms/step - loss: 0.0404 - val_loss: 0.0403
Epoch 143/200
mean max of probabilities: 0.00011207371 - temperature 0.14255308
 96/160 [=================>............] - ETA: 4s - loss: 0.0410